In [1]:
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

In [3]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv(dotenv_path="../../.env")
api_key = os.getenv("API_KEY")

In [4]:
embeddings_model = OpenAIEmbeddings(api_key=api_key)
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})
from langchain.memory import VectorStoreRetrieverMemory
# 리트리버 설정: k=1로 가장 관련성 높은 1개 문서를 반환하도록 지정
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})


# VectorStoreRetrieverMemory 객체 생성
memory = VectorStoreRetrieverMemory(retriever=retriever)
# 임의의 대화 4턴 저장
memory.save_context(
    inputs={"human": "안녕하세요, 오늘 면접에 참석해 주셔서 감사합니다. 자기소개 부탁드립니다."},
    outputs={"ai": "안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다."},
)
memory.save_context(
    inputs={"human": "프로젝트에서 어떤 역할을 맡았나요?"},
    outputs={"ai": "제가 맡은 역할은 백엔드 개발자였습니다. 사용자 데이터 처리와 서버 로직 개발을 담당했으며, RESTful API를 구현하여 프런트엔드와의 통신을 담당했습니다. 또한, 데이터베이스 설계에도 참여했습니다."},
)
memory.save_context(
    inputs={"human": "팀 프로젝트에서 어려움을 겪었던 경험이 있다면..."},
    outputs={"ai": "프로젝트 초기에 의사소통 문제로 몇 가지 어려움이 있었지만, 정기적인 팀 회의와 명확한 역할 분담을 통해 이를 극복하고 성공적으로 프로젝트를 완료했습니다."},
)
memory.save_context(
    inputs={"human": "개발자로서 자신의 강점은 무엇이라고 생각하나요?"},
    outputs={"ai": "제 강점은 빠른 학습 능력과 문제 해결 능력입니다. 새로운 기술을 배우고 적용하는 데 주저함이 없으며, 복잡한 문제에 직면했을 때 논리적으로 접근하여 해결책을 찾는 것을 즐깁니다."},
)

# 쿼리 구문을 통해 가장 관련성 높은 대화 1개 턴 검색
print(memory.load_memory_variables({"human": "면접자 전공은 무엇 인가요?"})["history"])
print("---")
print(memory.load_memory_variables({"human": "면접자가 프로젝트에서 맡은 역할은 무엇인가요?"})["history"])


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
/tmp/ipykernel_23175/310630537.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = VectorStoreRetrieverMemory(retriever=retriever)


human: 안녕하세요, 오늘 면접에 참석해 주셔서 감사합니다. 자기소개 부탁드립니다.
ai: 안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다.
---
human: 프로젝트에서 어떤 역할을 맡았나요?
ai: 제가 맡은 역할은 백엔드 개발자였습니다. 사용자 데이터 처리와 서버 로직 개발을 담당했으며, RESTful API를 구현하여 프런트엔드와의 통신을 담당했습니다. 또한, 데이터베이스 설계에도 참여했습니다.
